In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - LLaMA2 (PEFT Finetuning)

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_peft_finetuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_llama2_peft_finetuning.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_llama2_peft_finetuning.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
Open in Vertex AI Workbench
    </a> (A Python-3 CPU notebook is recommended)
  </td>
</table>

## Overview

This notebook demonstrates downloading [LLaMA2 models](https://huggingface.co/meta-llama), and finetuning and deploying LLaMA2 models with performance efficient finetuning libraries ([PEFT](https://github.com/huggingface/peft)) in Vertex AI. This notebook uses [Text moderation APIs](https://cloud.google.com/natural-language/docs/moderating-text) to analyze predictions against a list of safety attributes.


### Objective

- Download prebuilt LLaMA2 models
- Finetune and deploy LLaMA2 models with PEFT

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Cloud NL APIs

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), [Cloud NL API pricing](https://cloud.google.com/natural-language/pricing) and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## One-click fine-tuning and deployment

Vertex Model Garden offers a pre-configured fine-tuning pipeline that can be launched with just a few clicks in the Google Cloud Console. This Vertex AI pipeline will fine-tune, evaluate, upload and deploy your desired LLaMA2 model with an existing or custom dataset. 

Learn about [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)

### Create a new Vertex AI Pipeline run

1. Open the Google Cloud Console and navigate to Model Garden in Vertex AI.

2. Find the Llama 2 model card and press the `OPEN FINE-TUNING PIPELINE` button at the top. You will be redirected to a new Vertex AI Pipeline Run page.

3. Select `CREATE RUN` to open the run creation page.

4. Set a run name, region, run schedule, then press `CONTINUE` to the Runtime configuration step.

5. Set a Google Cloud Storage directory where pipeline artifacts will be stored.

6. Modify the pipeline parameter values as appropriate or keep the default values and press `SUBMIT`. The pipeline will begin running. Expect the run to take 45 minutes - 1 hour for Llama2-7b and longer for larger base models.

### Pipeline parameters

- `base_model`: The LLaMA2 base model (Llama2-7b, Llama2-7b-chat, Llama2-13b, Llama2-13b-chat, Llama2-70b, Llama2-70b-chat).

- `dataset_column`: The training and evaluation dataset column name. 

- `dataset_name`: The huggingface dataset name or gs:// URI path to a custom JSONL dataset.

- `evaluation_limit`: The limit of the number of examples per task. If set to a value less than 1.0, the limit is a percentage of the total number of examples.

- `evaluation_split_name`: Evaluation split name in the dataset.

- `evaluation_steps`: The number of training steps between evaluations.

- `prediction_accelerator_type`: The accelerator the model serving container is configured with (NVIDIA_L4, NVIDIA_TESLA_V100, NVIDIA_TESLA_A100).

- `prediction_docker_uri`: Artifact Registry URI of the docker image of the model serving container.

- `training_accelerator_type`: The accelerator the training job is configured with (NVIDIA_L4, NVIDIA_TESLA_V100, NVIDIA_TESLA_A100).

- `training_docker_uri`: Artifact Registry URI of the docker image of the training container.

- `training_learning_rate`: The learning rate after the potential warmup period.

- `training_lora_alpha`: LoRA scaling factor.

- `training_lora_dropout`: Dropout probability of the LoRA layers. 

- `training_lora_rank`: The rank of the update matrices, expressed in int. A lower rank results in smaller update matrices with fewer trainable parameters.

- `training_precision_mode`: Finetuning model precision mode (4bit, float16).

- `training_split_name`: The training split name.

- `training_steps`: The total number of training steps.

- `training_warmup_steps`: The numner of steps for the warmup in the learning rate scheduler.

Learn more about [LoRA](https://huggingface.co/docs/peft/conceptual_guides/lora) and how to set LoRA-specific parameters.

### Fine-tuning pipeline tasks

The fine-tuning pipeline is configured as an execution graph consisting of tasks as nodes. Each task runs as a Vertex AI Custom Job.

1. `validate-pipeline`: Validates input parameters and sets appropriate machine spec configurations.

2. `create-tensorboard-instance`: Creates a Vertex AI TensorBoard instance that will ingest tunings logs.

3. `compose-params-for-language-modeling-training-job`: Sets the input parameters for the tuning job.

4. `large-language-model-tuner`: Tunes the model and runs an evaluation. 

5. `upload-model`: Uploads the fine-tuned model to Vertex AI Model Registry.

6. `create-endpoint-and-deploy-model`: Creates a Vertex AI Endpoint and deploys the model for serving.

You can use the right-hand sidebar to view the input and output parameters of each task. You can also select the `VIEW JOB` button to open the Custom Job page. Use the `VIEW LOGS` button to see detailed logging during the job execution. 

### TensorBoard

A Vertex AI TensorBoard instance is automatically created during the pipeline run. During the tuning task, training and evaluation logs will stream to this TensorBoard. You can open the TensorBoard by selecting the training task node in the pipeline run UI and then selecting the `OPEN TENSORBOARD` link at the top of the Node Info tab.

Learn more about [Vertex AI TensorBoard.](https://cloud.google.com/vertex-ai/docs/experiments/tensorboard-introduction)

### Machine resource requirements

The fine-tuning pipeline requires adequate available machine resources to run successfully. Depending on the `base_model`, `prediction_accelerator_type` and `training_accelerator_type` specified, the pipeline will automatically configure a valid machine type and sufficient accelerator count for each job. If the run fails with a stockout or RESOURCE_EXHAUSTED error, you can specify different accelerators or a smaller `base_model` and retry the pipeline run. Alternatively, you can check your project quota for the requested compute resources and request an increase if it is not sufficient to run the training or deployment jobs. 

### Model Serving

Once the pipeline run finishes, your fine-tuned model will be deployed to a Vertex AI endpoint. In the run UI, select the `create-endpoint-and-deploy-model` task to open the Node Info tab on the right. Find the `endpoint_resource_name` in the Output Parameters section. See the `Send a prediction request` section below for an example on sending requests to this endpoint. 

## Before you begin

### Colab only
Run the following commands for Colab and skip this section if you are using Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    ! pip3 install --upgrade google-cloud-aiplatform
    ! pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

    # Restart the notebook kernel after installs.
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

### Workbench only
If you are using Workbench, you should find that the necessary dependencies are already pre-installed. If this is not the case or if you have previously modified the existing libraries, you may install the dependencies using the following commands:
```
! pip3 install --upgrade google-cloud-aiplatform
! pip3 install ipython pandas[output_formatting] google-cloud-language==2.10.0
```

### Setup Google Cloud project

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the Vertex AI API, Compute Engine API and Cloud Natural Language API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,language.googleapis.com).

1. [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs.

1. [Create a service account](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console) with `Vertex AI User` and `Storage Object Admin` roles for deploying fine tuned model to Vertex AI endpoint.

### Import the necessary packages

In [ ]:
import os
import sys
from datetime import datetime
from typing import Tuple

from google.cloud import aiplatform, language

Set the following variables for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the specified region (`REGION`). Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1").

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# Region for launching jobs.
REGION = ""  # @param {type:"string"}

# Cloud Storage bucket for storing experiments output.
# Start with gs:// prefix, e.g. gs://foo_bucket.
BUCKET_URI = "gs://"  # @param {type:"string"}

! gcloud config set project $PROJECT_ID
! gcloud services enable language.googleapis.com


STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
EXPERIMENT_BUCKET = os.path.join(BUCKET_URI, "peft")
BASE_MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "base_model")
MODEL_BUCKET = os.path.join(EXPERIMENT_BUCKET, "model")

# The service account looks like:
# '@.iam.gserviceaccount.com'
# Please go to https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console
# and create service account with `Vertex AI User` and `Storage Object Admin` roles.
# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user(project_id=PROJECT_ID)

### Initialize Vertex AI API

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

### Define constants

In [ ]:
# The pre-built training and serving docker images.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-train:20231130_0936_RC00"
PREDICTION_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-peft-serve:20231129_0948_RC00"

### Define common functions

In [ ]:
def get_job_name_with_datetime(prefix: str) -> str:
    """Gets the job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")


def deploy_model(
    model_name: str,
    base_model_id: str,
    finetuned_lora_model_path: str,
    service_account: str,
    task: str,
    precision_loading_mode: str = "float16",
    machine_type: str = "n1-standard-8",
    accelerator_type: str = "NVIDIA_TESLA_V100",
    accelerator_count: int = 1,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Deploys trained models into Vertex AI."""
    endpoint = aiplatform.Endpoint.create(display_name=f"{model_name}-endpoint")
    serving_env = {
        "BASE_MODEL_ID": base_model_id,
        "PRECISION_LOADING_MODE": precision_loading_mode,
        "TASK": task,
    }
    if finetuned_lora_model_path:
        serving_env["FINETUNED_LORA_MODEL_PATH"] = finetuned_lora_model_path
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=PREDICTION_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/peft_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=service_account,
    )
    return model, endpoint


def moderate_text(text: str) -> language.ModerateTextResponse:
    """Calls Vertex AI APIs to analyze text moderations."""
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.moderate_text(document=document)


def show_text_moderation(text: str, response: language.ModerateTextResponse) -> None:
    """Shows text moderation results."""
    import pandas as pd

 def confidence(category: language.ClassificationCategory) -> float:
        return category.confidence

    columns = ["category", "confidence"]
    categories = sorted(response.moderation_categories, key=confidence, reverse=True)
    data = ((category.name, category.confidence) for category in categories)
    df = pd.DataFrame(columns=columns, data=data)

    print(f"Text analyzed:\n{text}")
    print(df.to_markdown(index=False, tablefmt="presto", floatfmt=".0%"))

## Access LLaMA2 pretrained and finetuned models
The original models from Meta are converted into the Hugging Face format for finetuning and serving in Vertex AI.

Accept the model agreement to access the models:
1. Navigate to the Vertex AI > Model Garden page in the Google Cloud console
2. Find the LLaMA2 model card and click on "VIEW DETAILS"
3. Review the agreement on the model card page
4. After clicking the agreement of LLaMA2, a Cloud Storage bucket containing LLaMA2 pretrained and finetuned models will be shared
5. Paste the Cloud Storage bucket link below and assign it to `VERTEX_AI_MODEL_GARDEN_LLAMA2`

In [ ]:
VERTEX_AI_MODEL_GARDEN_LLAMA2 = ""  # This will be shared once click the agreement of LLaMA2 in Vertex AI Model Garden.
assert (
    VERTEX_AI_MODEL_GARDEN_LLAMA2
), "Please click the agreement of LLaMA2 in Vertex AI Model Garden, and get the GCS path of LLaMA2 model artifacts."
print(
    "Copy LLaMA2 model artifacts from",
    VERTEX_AI_MODEL_GARDEN_LLAMA2,
    "to ",
    BASE_MODEL_BUCKET,
)
! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_LLAMA2/* $BASE_MODEL_BUCKET

Set the base model id.

In [ ]:
base_model_name = "llama2-7b-chat-hf"  # @param ["llama2-7b-hf", "llama2-7b-chat-hf", "llama2-13b-hf", "llama2-13b-chat-hf", "llama2-70b-hf", "llama2-70b-chat-hf"]
base_model_id = os.path.join(BASE_MODEL_BUCKET, base_model_name)

## Finetune with Vertex AI Pipelines

Vertex Model Garden offers a pre-configured pipeline that can be launched with a single API command. This Vertex AI pipeline will fine-tune, evaluate, upload and deploy your desired LLaMA2 model.

Learn about [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction).

In [ ]:
# Set the path to the preconfigured Vertex AI Pipeline. The URI is the location of the pipeline template stored in an Artifact Registry. 
COMPILED_PIPELINE_PATH="https://us-kfp.pkg.dev/ml-pipeline/google-cloud-registry/oss-peft-llm-tuner/sha256:b359e8bc62a3aa97529c6a8c3e4c1e7e2bc94b9a8b6a037f048a649d5880312e"

# The LLaMA2 base model. Choose from "Llama-2-7b", "Llama-2-7b-chat", "Llama-2-13b", "Llama-2-13b-chat", "Llama-2-70b", "Llama-2-70b-chat"
base_model = 'Llama-2-7b'

# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = 'timdettmers/openassistant-guanaco'

# Sets the accelerator type to train and deploy the finetuned model.
# Acceptable accelerator types are "NVIDIA_L4", "NVIDIA_TESLA_V100", "NVIDIA_TESLA_A100".
# The pipeline will automatically configure a sufficient accelerator count and machine type based on your chosen base model and accelerator types.
prediction_accelerator_type = 'NVIDIA_L4'
training_accelerator_type = 'NVIDIA_TESLA_V100'

# The supported precision loading types are "4bit" and "float16".
training_precision_mode = '4bit'

pipeline_parameters = {
    'base_model': base_model,
    'dataset_name': dataset_name,
    'prediction_accelerator_type': prediction_accelerator_type,
    'training_accelerator_type': training_accelerator_type,
    'training_precision_mode': training_precision_mode,
    'training_lora_rank':16,
    'training_lora_alpha':32,
    'training_lora_dropout': 0.05,
    'training_steps': 20,
    'training_warmup_steps': 10,
    'training_learning_rate': 2e-4,
    'evaluation_steps': 10,
    'evaluation_limit': 100,
}

pipeline_bucket_uri = os.path.join(BUCKET_URI, "pipeline_runs")

# Define and launch the Pipeline Job. 
job = aiplatform.PipelineJob(display_name=get_job_name_with_datetime(prefix="llama2-peft"),
                             template_path=COMPILED_PIPELINE_PATH,
                             pipeline_root=pipeline_bucket_uri,
                             parameter_values=pipeline_parameters)

job.submit(service_account=SERVICE_ACCOUNT)

You can monitor the pipeline run by navigating to the "Vertex AI Pipelines" page in the Google Cloud Console and selecting your pipeline run. This pipeline job will take up to 1 hour to complete for the LLaMA2-7B model.

Once the pipeline run finishes, your fine-tuned model will be deployed to a Vertex AI endpoint. In the run UI, select the `create-endpoint-and-deploy-model` task to open the Node Info tab on the right. Find the `endpoint_resource_name` in the Output Parameters section. See the `Send a prediction request` section below for an example on sending requests to this endpoint.

## Finetune and deploy LLaMA2 models with PEFT

This section demonstrates how to finetune and deploy LLaMA2 models with PEFT LoRA. LoRA (Low-Rank Adaptation) is one approach of PEFT (Parameter Efficient FineTuning), where pretrained model weights are frozen and rank decomposition matrices representing the change in model weights are trained during finetuning. Read more about LoRA in the following publication: [Hu, E.J., Shen, Y., Wallis, P., Allen-Zhu, Z., Li, Y., Wang, S., Wang, L. and Chen, W., 2021. Lora: Low-rank adaptation of large language models. *arXiv preprint arXiv:2106.09685*](https://arxiv.org/abs/2106.09685).

### Finetune

Use the Vertex AI SDK to create and run the custom training jobs with Vertex AI Model Garden training images.

This example uses the dataset [Abirate/english_quotes](https://huggingface.co/datasets/Abirate/english_quotes). You can either use a [dataset from huggingface](https://huggingface.co/datasets) or a custom JSONL dataset in [Vertex text model dataset format](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format) stored in Cloud Storage. The `template` parameter is optional.

#### [Optional] Finetune with a custom dataset

To use a custom dataset, you should supply a `gs://` URI to a JSONL file in [Vertex text model dataset format](https://cloud.google.com/vertex-ai/docs/generative-ai/models/tune-text-models-supervised#dataset-format) in the `dataset_name` below.

For example, here is one data point from the sample dataset `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl`:

```json
{"input_text":"TRANSCRIPT: \nREASON FOR EVALUATION:,\n\n LABEL:","output_text":"Chiropractic"}
```

To use this sample dataset that contains `input_text` and `output_text` fields, set `dataset_name` to `gs://cloud-samples-data/vertex-ai/model-evaluation/peft_train_sample.jsonl` and `template` to `vertex_sample`. For advanced usage with custom datatset fields, see [the template example](https://github.com/tloen/alpaca-lora/blob/main/templates/alpaca.json) and supply your own JSON template as `gs://` URIs.

In [ ]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "Abirate/english_quotes"  # @param {type:"string"}
# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

finetuning_precision_mode = "float16"

# Worker pool spec.
# Finetunes LLaMA2 7B with 1 V100 (16G).
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1

# Finetunes LLaMA2 7B with 1 L4 (24G).
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# If A100 is not available, you may finetune LLaMA2 13B models with multiple
# V100s or L4s. Please keep in mind that the efficiency of finetuning with
# multiple V100s or L4s is inferior to finetuning with 1 A100.
# Finetunes LLaMA2 13B with 2 V100 (16G).
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2

# Finetunes LLaMA2 13B with 2 L4 (24G).
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

# Finetunes LLaMA2 13B with 1 A100 (40G).
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Finetunes LLaMA2 70B with 8 L4 (24G).
# machine_type = "g2-standard-96"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 8

# Finetunes LLaMA2 70B with 4 A100 (40G).
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4

replica_count = 1


# Setup training job.
job_name = get_job_name_with_datetime("llama2-lora-train")
output_dir = os.path.join(MODEL_BUCKET, job_name)
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")

Run the following cell to launch the training job; Or skip to the next section if you want to try hyperparameter tuning with Vertex AI.

In [ ]:
# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
)
train_job.run(
    args=[
        "--task=causal-language-modeling-lora",
        f"--pretrained_model_id={base_model_id}",
        f"--dataset_name={dataset_name}",
        f"--output_dir={output_dir}",
        "--lora_rank=16",
        "--lora_alpha=32",
        "--lora_dropout=0.05",
        "--warmup_steps=10",
        "--max_steps=10",
        "--learning_rate=2e-4",
        f"--precision_mode={finetuning_precision_mode}",
        f"--template={template}",
    ],
    environment_variables={"WANDB_DISABLED": True},
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
)

print("Trained models were saved in: ", output_dir)

### [Optional] Hyperparameter tuning

You can use the Vertex AI SDK to create and run the [hyperparameter tuning job](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) to obtain a better performance by experimenting with different hyperparameters such as learning rates.

Define the following specifications:

- `worker_pool_specs`: Dictionary specifying the machine type and Docker image.

- `parameter_spec`: Dictionary specifying the parameters to optimize. The dictionary key is the string assigned to the command line argument for each hyperparameter in your training application code, and the dictionary value is the parameter specification. The parameter specification includes the type, min/max values, and scale for the hyperparameter.

- `metric_spec`: Dictionary specifying the metric to optimize. The dictionary key is the hyperparameter_metric_tag that you set in your training application code, and the value is the optimization goal.

The following 4bit QLoRA experiment results show the effectiveness of hyperparameter tuning evaluated on the ARC Challenge dataset (for reference only):

| Model      | Training time | Trials | Parallel Trials | GPU  | ∆arc challenge | ∆hellaswag | ∆truthfulqa_mc | cost        |
|------------|---------------|--------|-----------------|------|----------------|------------|----------------|-------------|
| Llama2-7b  | 2d 10hrs      | 8      | 1               | L4x1 | +0.73          | +1.61      | +5.34          | \$49.5088    |
| Llama2-13b | 4d 8hrs       | 8      | 1               | L4x1 | +1.53          | +2.11      | +10.98         | \$88.7744    |
| Llama2-70b | 6d 10hrs      | 8      | 2               | L4x4 | +0.77          | +0.93      | +10.63         | \$1,312.5461 |

The following example runs 8 trials on `timdettmers/openassistant-guanaco` with different learning rates, and evaluates the model on `arc_challenge` dataset. You can customize the search space by extending the range of learning rates, adding other parameters such as LoRA rank, etc. Please refer to the [hyperparameter tuning documentation](https://cloud.google.com/vertex-ai/docs/training/hyperparameter-tuning-overview) for more information.

In [ ]:
# Huggingface dataset name or gs:// URI to a custom JSONL dataset.
dataset_name = "timdettmers/openassistant-guanaco"  # @param {type:"string"}
# Optional. Template name or gs:// URI to a custom template.
template = ""  # @param {type:"string"}

hpt_precision_mode = "4bit"

# Worker pool spec for 4bit finetuning.

# Finetunes LLaMA2 7B with 1 L4 (24G).
machine_type = "g2-standard-8"
accelerator_type = "NVIDIA_L4"
accelerator_count = 1

# Finetunes LLaMA2 13B with 1 L4 (24G).
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# Finetunes LLaMA2 70B with 4 L4 (24G).
# machine_type = "g2-standard-48"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 4

### [Optional] Custom evaluation dataset

To obtain a model with better performance on some specific tasks, you might want to run hyperparameter tuning with a custom evaluation dataset. The hyperparameter tuning service will pick the model according to the evaluation dataset and the metrics you selected. You can use any of the following tasks as the `eval_task` in the code cell below:

1. The name of a [lm-evaluation-harness task](https://github.com/EleutherAI/lm-evaluation-harness/tree/big-refactor/lm_eval/tasks).

2. `custom_likelihood`. Then, add a flag `--eval_dataset_path=<Cloud Storage URI to your JSONL dataset>`. The JSONL file must be in the format in Vertex AI language model's [prepare evaluation dataset](https://cloud.google.com/vertex-ai/docs/generative-ai/models/evaluate-models#classification) page.

3. `builtin_eval`. The built-in evaluation loop of the trainer will be used to evaluate the model instead of the [lm-evaluation-harness](https://github.com/EleutherAI/lm-evaluation-harness) library. You can supply any eval dataset in the same format as the training dataset by specifying `--eval_dataset_path`, `--eval_split`, `--eval_template`, and `--eval_column`.

In [ ]:
from google.cloud.aiplatform import hyperparameter_tuning as hpt

eval_task = "arc_challenge"  # @param {type:"string"}
eval_metric_name = "acc_norm"  # @param {type:"string"}

# Runs 10 training steps as a minimal example. Use 1000 to reproduce the experiment results.
max_steps = 10  # @param {type:"integer"}
# Evaluates the model on 10 examples. Use 10000 to reproduce the experiment results.
eval_limit = 10  # @param {type:"integer"}

flags = {
    "learning_rate": 1e-5,
    "precision_mode": hpt_precision_mode,
    "task": "instruct-lora",
    "pretrained_model_id": base_model_id,
    "output_dir": output_dir,
    "warmup_steps": 10,
    "max_steps": max_steps,
    "lora_rank": 32,
    "lora_alpha": 64,
    "lora_dropout": 0.05,
    "dataset_name": dataset_name,
    "eval_steps": max_steps + 1,  # Only evaluates in the end.
    "eval_tasks": eval_task,
    "eval_limit": eval_limit,
    "eval_metric_name": eval_metric_name,
}

worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": machine_type,
            "accelerator_type": accelerator_type,
            "accelerator_count": accelerator_count,
        },
        "replica_count": replica_count,
        "container_spec": {
            "image_uri": TRAIN_DOCKER_URI,
            "args": ["--{}={}".format(k, v) for k, v in flags.items()],
        },
    }
]
metric_spec = {"model_performance": "maximize"}
parameter_spec = {
    "learning_rate": hpt.DoubleParameterSpec(min=1e-5, max=1e-4, scale="linear"),
}
train_job = aiplatform.CustomJob(
    display_name=job_name,
    worker_pool_specs=worker_pool_specs,
    staging_bucket=STAGING_BUCKET,
)

train_hpt_job = aiplatform.HyperparameterTuningJob(
    display_name=f"{job_name}_hpt",
    custom_job=train_job,
    metric_spec=metric_spec,
    parameter_spec=parameter_spec,
    max_trial_count=8,
    parallel_trial_count=2,
)

train_hpt_job.run()

print("Trained models were saved in: ", output_dir)

Then, find the best trial from the hyperparameter tuning job.

In [ ]:
best_trial_id = max(
    train_hpt_job.trials, key=lambda trial: trial.final_measurement.metrics[0].value
).id
output_dir = os.path.join(output_dir, f"trial_{best_trial_id}")
output_dir_gcsfuse = output_dir.replace("gs://", "/gcs/")
print(f"Best trial {best_trial_id} saved model in:", output_dir)

### Deploy fine tuned models with Google Cloud Text Moderation
This section uploads the model to Model Registry and deploys it on the Endpoint.

The model deployment step will take 15 minutes to 1 hour to complete, depending on the model sizes.

In [ ]:
# Finds Vertex AI prediction supported accelerators and regions in
# https://cloud.google.com/vertex-ai/docs/predictions/configure-compute.

# Sets V100 (16G) to deploy LLaMA2 7B models.
# V100 serving has better throughput and latency performance than L4 serving.
machine_type = "n1-standard-8"
accelerator_type = "NVIDIA_TESLA_V100"
accelerator_count = 1

# Sets 1 L4 (24G) to deploy LLaMA2 7B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-8"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 1

# If A100 is not available, you may serve LLaMA2 13B models with multiple V100s
# or L4s. Please keep in mind that the efficiency of serving with multiple
# V100s or L4s is inferior to serving with 1 A100.
# Sets 2 V100 (16G) to deploy LLaMA2 13B models.
# V100 serving has better throughput and latency performance than L4 serving.
# machine_type = "n1-standard-8"
# accelerator_type = "NVIDIA_TESLA_V100"
# accelerator_count = 2

# Sets 2 L4 (24G) to deploy LLaMA2 13B models.
# L4 serving is more cost efficient than V100 serving.
# machine_type = "g2-standard-24"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 2

# Sets A100 (40G) to deploy LLaMA2 13B models.
# machine_type = "a2-highgpu-1g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 1

# Sets 8 L4 (24G) to deploy LLaMA2 70B models.
# If you do not have access to 4 A100 (40G) GPUs, you may serve LLaMA 2 70B
# models with 8 L4 (24G) GPUs.
# Note that with the default timeout threshold of Vertex endpoints, you should
# set a `max_length` configuration of around 1,000 tokens or fewer. If you need
# longer generated sequences, please file a request with Vertex to allowlist
# your project for a longer timeout threshold with Vertex endpoints.
# machine_type = "g2-standard-96"
# accelerator_type = "NVIDIA_L4"
# accelerator_count = 8

# Sets 4 A100 (40G) to deploy LLaMA2 70B models.
# machine_type = "a2-highgpu-4g"
# accelerator_type = "NVIDIA_TESLA_A100"
# accelerator_count = 4

# The supported precision loading types are "4bit", "8bit", "float16" and "float32".
precision_loading_mode = finetuning_precision_mode

model_with_peft, endpoint_with_peft = deploy_model(
    model_name=get_job_name_with_datetime(prefix="llama2-peft-serve"),
    base_model_id=base_model_id,
    finetuned_lora_model_path=output_dir,
    service_account=SERVICE_ACCOUNT,
    task="causal-language-modeling-lora",
    precision_loading_mode=precision_loading_mode,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
)
print("endpoint_name:", endpoint_with_peft.name)

NOTE: After the deployment succeeds, the base model weights will be downloaded on the fly from $BASE_MODEL_BUCKET and LoRA model weights will be downloaded from the GCS bucket used in training above. Thus additional 10 ~ 40 minutes (depending on the model sizes) of waiting time is needed **after** the above model deployment step succeeds and before you run the next step below. Otherwise you might see a `ServiceUnavailable: 503 502:Bad Gateway` error when you send requests to the endpoint.

### Send a prediction request

Once deployment succeeds, you can send requests to the endpoint with text prompts.

Example:

```
Human: What is a car?
Assistant:  A car, or a motor car, is a road-connected human-transportation system used to move people or goods from one place to another. The term also encompasses a wide range of vehicles, including motorboats, trains, and aircrafts. Cars typically have four wheels, a cabin for passengers, and an engine or motor. They have been around since the early 19th century and are now one of the most popular forms of transportation, used for daily commuting, shopping, and other purposes.
```

In [ ]:
# Loads an existing endpoint instance using the endpoint name:
# - Using `endpoint_name = endpoint_with_peft.name` allows us to get the
#   endpoint name of the endpoint `endpoint_with_peft` created in the cell
#   above.
# - Alternatively, you can set `endpoint_name = "1234567890123456789"` to load
#   an existing endpoint with the ID 1234567890123456789.
# You may uncomment the code below to load an existing endpoint.

# endpoint_name = endpoint_with_peft.name
# # endpoint_name = ""  # @param {type:"string"}
# aip_endpoint_name = (
#     f"projects/{PROJECT_ID}/locations/{REGION}/endpoints/{endpoint_name}"
# )
# endpoint_with_peft = aiplatform.Endpoint(aip_endpoint_name)


# Overides max_length and top_k parameters during inferences.
# If you encounter the issue like `ServiceUnavailable: 503 Took too long to respond when processing`,
# you can reduce the max length, such as set max_length as 20.
# If you are using L4 GPUs to serve LLaMA2 70B models, you should set
# max_length to around 1,000 tokens or fewer. If you need longer generated
# sequences, please file a request with Vertex to allowlist your project for a
# longer timeout threshold with Vertex endpoints.
instances = [
    {
        "prompt": "What is a car?",
        "max_tokens": 50,
        "temperature": 1.0,
        "top_p": 1.0,
        "top_k": 10,
    },
]
response = endpoint_with_peft.predict(instances=instances)

for prediction in response.predictions:
    print(prediction)

Text moderation analyzes a document against a list of safety attributes, which include "harmful categories" and topics that may be considered sensitive.

In [ ]:
for generated_text in response.predictions:
    # Send a request to the API.
    response = moderate_text(generated_text)
    # Show the results.
    show_text_moderation(generated_text, response)

## Clean up resources

In [ ]:
if train_job._gca_resource.name:
    # Training job is submitted.
    train_job.delete()
# Uncomment the following line to delete the hyperparameter tuning job if you
# have created one.
# train_hpt_job.delete()

# Undeploy model and delete endpoint.
endpoint_with_peft.delete(force=True)

# Delete models.
model_with_peft.delete()

# Delete Cloud Storage objects that were created
delete_bucket = False
if delete_bucket:
    ! gsutil -m rm -r $EXPERIMENT_BUCKET